In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
FOLDERNAME = 'Colab\ Notebooks/3_Split'

In [ ]:
%cd drive/MyDrive/$FOLDERNAME

/content/drive/MyDrive/Colab Notebooks/3_Split


In [ ]:
# %ls

In [ ]:
import os
import glob
import shutil
import pandas as pd
from tqdm.notebook import tqdm, trange

In [ ]:
files = glob.glob("Train/*.jpg")
len(files)

1949

In [ ]:
def _open(path):
  with open(path, 'r') as f:
    label=f.read()
    return label

In [ ]:
df = pd.DataFrame(
    dict(
        subset=[f.split("/")[0] for f in files],
        image_path=files,
        # category_path = [f.replace(".jpg", ".txt") for f in files],  
        category = [_open(f.replace(".jpg", ".txt")) for f in tqdm(files)],
    )
)
df

  0%|          | 0/1949 [00:00<?, ?it/s]

,subset,image_path,category
0,Train,Train/b_725.jpg,ICH
1,Train,Train/b_737.jpg,ICH
2,Train,Train/b_1254.jpg,ICH
3,Train,Train/b_522.jpg,ICH
4,Train,Train/b_1251.jpg,ICH
...,...,...,...
1944,Train,Train/b_1142.jpg,Ischemic
1945,Train,Train/b_980.jpg,Ischemic
1946,Train,Train/b_1000.jpg,Ischemic
1947,Train,Train/b_878.jpg,Ischemic


In [ ]:
# print(_open(df['category'][0]))

FileNotFoundError: ignored

In [ ]:
df.groupby(by=['subset', 'category']).count()
# df['category'].dtype

image_path
subset category              
Train  ICH                240
       Ischemic          1117
       Non-stroke         592

In [ ]:
# df['_subset'] = df['subset'].to_string

In [ ]:
df_oversample = pd.concat(
    [
      df.query("subset == 'Train' and category == 'Ischemic'"),
      df.query("subset == 'Train' and category == 'ICH'").sample(frac=1675/240, replace=True),
      df.query("subset == 'Train' and category == 'Non-stroke'").sample(frac=1117/592, replace=True),
    ],
    axis=0,
).reset_index(drop=True)

In [ ]:
df_oversample

,subset,image_path,category
0,Train,Train/b_1107.jpg,Ischemic
1,Train,Train/b_580.jpg,Ischemic
2,Train,Train/b_356.jpg,Ischemic
3,Train,Train/b_890.jpg,Ischemic
4,Train,Train/b_541.jpg,Ischemic
...,...,...,...
3904,Train,Train/b_2860.jpg,Non-stroke
3905,Train,Train/b_2729.jpg,Non-stroke
3906,Train,Train/b_3286.jpg,Non-stroke
3907,Train,Train/b_2986.jpg,Non-stroke


In [ ]:
df_oversample.to_csv('df_oversample2.csv')




In [1]:
df_oversample = pd.read_csv('df_oversample2.csv')
df_oversample

NameError: ignored

In [ ]:
os.makedirs("Train_oversample2", exist_ok=True)

for i in tqdm(range(len(df_oversample))):
  img_path, category = df_oversample.iloc[i][["image_path", "category"]]
  img_new_path = os.path.join('Train_oversample2', str(i)+'.jpg')
  shutil.copy(img_path, img_new_path)
  label_new_path = img_new_path.replace(".jpg", ".txt")
  with open(label_new_path, 'w') as f:
    f.write(category)

  0%|          | 0/3909 [00:00<?, ?it/s]

In [ ]:
files_jpg = glob.glob("Train_oversample2/*.jpg")
len(files_jpg)

3909

In [ ]:
files_txt = glob.glob("Train_oversample2/*.txt")
len(files_txt)

3909

In [ ]:
with open('Train_oversample/3322.txt', 'r') as f:
  label = f.read()
label

'Non-stroke'